# DDRNet_with_hardnet_bottle2nek


Вместо стандартных блоков, предлагаемых в [DDRNet](https://github.com/chenjun2hao/DDRNet.pytorch/blob/1daf52a3eeb638f86fd07d2fba3a46c68bca8f15/lib/models/ddrnet_23_slim.py#L203-L342), используется [HarDBlock(4 слоя)](https://github.com/PingoLH/FCHarDNet/blob/fc4b854b5cfa01a449bcfaece6bb3c32d84d9e2b/ptsemseg/models/hardnet.py#L169-L233)

Также, вместо стандартного "бутылочного горлышка"(Bottleneck), используется [Bottle2neck](https://github.com/Res2Net/Res2Net-ImageNet-Training/blob/d77c16ff111522c64e918900f100699acc62f706/res2net_pami/res2net.py#L20-L96) из Res2net

In [ ]:
import sys
sys.path.insert(0, '.')
import os
import logging
import random
import time
import math
import torch
import numpy as np 
import torch.nn as nn
import torch.nn.functional as F

from torch.nn import init
from torch.utils import data
from datetime import datetime
from collections import OrderedDict

from torchsummary import summary
from torch.optim.lr_scheduler import _LRScheduler

import torch.distributed as dist

 
!pip install tensorboardX
from tensorboardX import SummaryWriter

     |████████████████████████████████| 122kB 8.3MB/s 


In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
max_iter = 120000
batch_size = 8 if torch.cuda.get_device_properties(0).total_memory/2**30 > 13 else 4
n_workers = 2
print_interval=10
val_interval=500

model_arch = 'ddrnet_slim_23'


BatchNorm2d = nn.BatchNorm2d
bn_mom = 0.1

In [ ]:
batch_size

8

## Blocks

In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, no_relu=False):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = BatchNorm2d(planes, momentum=bn_mom)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = BatchNorm2d(planes, momentum=bn_mom)
        self.downsample = downsample
        self.stride = stride
        self.no_relu = no_relu

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual

        if self.no_relu:
            return out
        else:
            return self.relu(out)

In [ ]:
class Bottleneck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None, no_relu=True):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = BatchNorm2d(planes, momentum=bn_mom)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = BatchNorm2d(planes, momentum=bn_mom)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1,
                               bias=False)
        self.bn3 = BatchNorm2d(planes * self.expansion, momentum=bn_mom)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride
        self.no_relu = no_relu

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        if self.no_relu:
            return out
        else:
            return self.relu(out)

In [ ]:
class Bottle2neck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None, baseWidth=26, scale = 4, stype='normal'):
        """ Constructor
        Args:
            inplanes: input channel dimensionality
            planes: output channel dimensionality
            stride: conv stride. Replaces pooling layer.
            downsample: None when stride = 1
            baseWidth: basic width of conv3x3
            scale: number of scale.
            type: 'normal': normal set. 'stage': first block of a new stage.
        """
        super(Bottle2neck, self).__init__()

        width = int(math.floor(planes * (baseWidth/64.0)))
        self.conv1 = nn.Conv2d(inplanes, width*scale, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width*scale)
        
        if scale == 1:
          self.nums = 1
        else:
          self.nums = scale -1
        if stype == 'stage':
            self.pool = nn.AvgPool2d(kernel_size=3, stride = stride, padding=1)
        convs = []
        bns = []
        for i in range(self.nums):
          convs.append(nn.Conv2d(width, width, kernel_size=3, stride = stride, padding=1, bias=False))
          bns.append(nn.BatchNorm2d(width))
        self.convs = nn.ModuleList(convs)
        self.bns = nn.ModuleList(bns)

        self.conv3 = nn.Conv2d(width*scale, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)

        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stype = stype
        self.scale = scale
        self.width  = width

    def forward(self, x):
        
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        spx = torch.split(out, self.width, 1)
        for i in range(self.nums):
          if i==0 or self.stype=='stage':
            sp = spx[i]
          else:
            sp = sp + spx[i]
          sp = self.convs[i](sp)
          sp = self.relu(self.bns[i](sp))
          if i==0:
            out = sp
          else:
            out = torch.cat((out, sp), 1)
        if self.scale != 1 and self.stype=='normal':
          out = torch.cat((out, spx[self.nums]),1)
        elif self.scale != 1 and self.stype=='stage':
          out = torch.cat((out, self.pool(spx[self.nums])),1)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

In [ ]:
class DAPPM(nn.Module):
    def __init__(self, inplanes, branch_planes, outplanes):
        super(DAPPM, self).__init__()
        self.scale1 = nn.Sequential(nn.AvgPool2d(kernel_size=5, stride=2, padding=2),
                                    BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale2 = nn.Sequential(nn.AvgPool2d(kernel_size=9, stride=4, padding=4),
                                    BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale3 = nn.Sequential(nn.AvgPool2d(kernel_size=17, stride=8, padding=8),
                                    BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale4 = nn.Sequential(nn.AdaptiveAvgPool2d((1, 1)),
                                    BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.scale0 = nn.Sequential(
                                    BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, branch_planes, kernel_size=1, bias=False),
                                    )
        self.process1 = nn.Sequential(
                                    BatchNorm2d(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.process2 = nn.Sequential(
                                    BatchNorm2d(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.process3 = nn.Sequential(
                                    BatchNorm2d(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )
        self.process4 = nn.Sequential(
                                    BatchNorm2d(branch_planes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes, branch_planes, kernel_size=3, padding=1, bias=False),
                                    )        
        self.compression = nn.Sequential(
                                    BatchNorm2d(branch_planes * 5, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(branch_planes * 5, outplanes, kernel_size=1, bias=False),
                                    )
        self.shortcut = nn.Sequential(
                                    BatchNorm2d(inplanes, momentum=bn_mom),
                                    nn.ReLU(inplace=True),
                                    nn.Conv2d(inplanes, outplanes, kernel_size=1, bias=False),
                                    )

    def forward(self, x):

        #x = self.downsample(x)
        width = x.shape[-1]
        height = x.shape[-2]
        x_list = []

        x_list.append(self.scale0(x))
        x_list.append(self.process1((F.interpolate(self.scale1(x),
                        size=[height, width],
                        mode='bilinear')+x_list[0])))
        x_list.append((self.process2((F.interpolate(self.scale2(x),
                        size=[height, width],
                        mode='bilinear')+x_list[1]))))
        x_list.append(self.process3((F.interpolate(self.scale3(x),
                        size=[height, width],
                        mode='bilinear')+x_list[2])))
        x_list.append(self.process4((F.interpolate(self.scale4(x),
                        size=[height, width],
                        mode='bilinear')+x_list[3])))
       
        out = self.compression(torch.cat(x_list, 1)) + self.shortcut(x)
        return out 

In [ ]:
class segmenthead(nn.Module):

    def __init__(self, inplanes, interplanes, outplanes, scale_factor=None):
        super(segmenthead, self).__init__()
        self.bn1 = BatchNorm2d(inplanes, momentum=bn_mom)
        self.conv1 = nn.Conv2d(inplanes, interplanes, kernel_size=3, padding=1, bias=False)
        self.bn2 = BatchNorm2d(interplanes, momentum=bn_mom)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(interplanes, outplanes, kernel_size=1, padding=0, bias=True)
        self.scale_factor = scale_factor

    def forward(self, x):
        
        x = self.conv1(self.relu(self.bn1(x)))
        out = self.conv2(self.relu(self.bn2(x)))

        if self.scale_factor is not None:
            height = x.shape[-2] * self.scale_factor
            width = x.shape[-1] * self.scale_factor
            out = F.interpolate(out,
                        size=[height, width],
                        mode='bilinear')

        return out

In [ ]:
class ConvLayer(nn.Sequential):
    def __init__(self, in_channels, out_channels, kernel=3, stride=1, dropout=0.1, padding=None, dilation=1):
        super().__init__()
        if padding is None:
            padding = kernel//2 + dilation - 1
        self.add_module('conv', nn.Conv2d(in_channels, out_channels, kernel_size=kernel,
                                          stride=stride, padding=padding, dilation=dilation, bias = False))
        self.add_module('norm', nn.BatchNorm2d(out_channels, momentum=bn_mom))
        self.add_module('relu', nn.ReLU(inplace=True))

    def forward(self, x):
        return super().forward(x)

In [ ]:
class HarDBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, grmul, n_layers, keepBase=False):
        super().__init__()
        self.in_channels = in_channels
        self.growth_rate = growth_rate
        self.grmul = grmul
        self.links = []
        self.out_channels = 0
        self.keepBase = keepBase
        self.layers = nn.ModuleList([])
        for i in range(n_layers):
            out_ch, in_ch, el_links = self.get_links(i+1)
            self.links.append(el_links)
            self.layers.append(ConvLayer(in_ch, out_ch))
            if (i % 2 == 0) or (i == n_layers - 1):
                self.out_channels += out_ch

    def __out_ch(self, layer_id): 
        out_ch = self.growth_rate
        for i in range(1, int(math.log2(layer_id))+1):
            if layer_id % 2**i == 0:
                out_ch = out_ch*self.grmul
        return int(int(out_ch + 1) / 2) * 2

    def get_links(self, layer_id):
        in_ch = 0
        links_ = []
        for i in range(int(math.log2(layer_id))):
            diff = 2**i
            if (layer_id % diff == 0) and layer_id - diff > 0:
                in_ch += self.__out_ch(layer_id - diff)
                links_.append(layer_id - diff)
        if math.log2(layer_id).is_integer():
            in_ch += self.in_channels
            links_.append(0)
        return self.__out_ch(layer_id), in_ch, links_

    def forward(self, x):
        data = [x]
        for layer in range(len(self.layers)):
            layer_input = []
            for link in self.links[layer]:
                layer_input.append(data[link])
            in_ = layer_input[0] if len(layer_input) == 1 \
                else torch.cat(layer_input, dim=1)
            data.append(self.layers[layer](in_))
        t = len(data)
        out = []
        for i in range(t):
            if (i % 2 == 1) or (self.keepBase and i == 0) or (i == t-1):
                out.append(data[i])
        return torch.cat(out, dim=1)
    
    def get_out_ch(self):
        return self.out_channels

## Model

In [ ]:
class DualResNet(nn.Module):

    def __init__(self, num_classes=19, planes=64, spp_planes=128, head_planes=128, augment=False):
        super(DualResNet, self).__init__()
        growth_rate = [  10,16,18,24,32]
        highres_planes = planes * 2
        self.augment = augment

        self.conv1 =  nn.Sequential(
                          nn.Conv2d(3,planes,kernel_size=3, stride=2, padding=1),
                          BatchNorm2d(planes, momentum=bn_mom),
                          nn.ReLU(inplace=True),
                          nn.Conv2d(planes,planes,kernel_size=3, stride=2, padding=1),
                          BatchNorm2d(planes, momentum=bn_mom),
                          nn.ReLU(inplace=True),
                      )

        self.relu = nn.ReLU(inplace=False)

        self.layer1 = self._make_layer_hardblock(planes,     planes,     stride=1, growth_rate=growth_rate[0])
        self.layer2 = self._make_layer_hardblock(planes,     planes * 2, stride=2, growth_rate=growth_rate[1])
        self.layer3 = self._make_layer_hardblock(planes * 2, planes * 4, stride=2, growth_rate=growth_rate[2])
        self.layer4 = self._make_layer_hardblock(planes * 4, planes * 8, stride=2, growth_rate=growth_rate[3])


        self.compression3 = nn.Sequential(
                                          nn.Conv2d(planes * 4, highres_planes, kernel_size=1, bias=False),
                                          BatchNorm2d(highres_planes, momentum=bn_mom),
                                          )

        self.compression4 = nn.Sequential(
                                          nn.Conv2d(planes * 8, highres_planes, kernel_size=1, bias=False),
                                          BatchNorm2d(highres_planes, momentum=bn_mom),
                                          )

        self.down3 = nn.Sequential(
                                   nn.Conv2d(highres_planes, planes * 4, kernel_size=3, stride=2, padding=1, bias=False),
                                   BatchNorm2d(planes * 4, momentum=bn_mom),
                                   )

        self.down4 = nn.Sequential(
                                   nn.Conv2d(highres_planes, planes * 4, kernel_size=3, stride=2, padding=1, bias=False),
                                   BatchNorm2d(planes * 4, momentum=bn_mom),
                                   nn.ReLU(inplace=True),
                                   nn.Conv2d(planes * 4, planes * 8, kernel_size=3, stride=2, padding=1, bias=False),
                                   BatchNorm2d(planes * 8, momentum=bn_mom),
                                   )

        self.layer3_ = self._make_layer_hardblock(planes * 2, highres_planes, stride=1, growth_rate=growth_rate[2])
        
        self.layer4_ = self._make_layer_hardblock(highres_planes, highres_planes, stride=1, growth_rate=growth_rate[2])

        self.layer5_ = self._make_bottle2neck(highres_planes, highres_planes)
        
        self.layer5 =  self._make_bottle2neck(planes * 8, planes * 8, stride=2)
        
        self.spp = DAPPM(planes * 8 * Bottle2neck.expansion, spp_planes, planes * 4) 
        
        if self.augment:
            self.seghead_extra = segmenthead(highres_planes, head_planes, num_classes)            

        self.final_layer = segmenthead(planes * 4, head_planes, num_classes)


        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)


    def _make_layer_hardblock(self, inplanes, planes, stride=1, growth_rate=10, grmul=1.7, n_layers=4):
        layers = []
        if stride != 1:
            layers.append(ConvLayer(inplanes, planes, kernel=1, stride=stride))
        hardblock_inplanes = planes if stride != 1 else inplanes
        hardblock = HarDBlock(hardblock_inplanes, growth_rate, grmul, n_layers)
        layers.append(hardblock)

        layers.append(ConvLayer(hardblock.get_out_ch(), planes, kernel=1, stride=1))
    
        return nn.Sequential(*layers)

    def _make_layer(self, block, inplanes, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, momentum=bn_mom),
            )

        layers = []
        layers.append(block(inplanes, planes, stride, downsample))
        inplanes = planes * block.expansion
        for i in range(1, blocks):
            if i == (blocks-1):
                layers.append(block(inplanes, planes, stride=1, no_relu=True))
            else:
                layers.append(block(inplanes, planes, stride=1, no_relu=False))

        return nn.Sequential(*layers)

    def _make_bottle2neck(self, inplanes, planes, stride=1):
        '''
            outplanes = planes * 4
        '''

        downsample = None
        if stride != 1 or inplanes != planes * Bottle2neck.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(inplanes, planes * Bottle2neck.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * Bottle2neck.expansion, momentum=bn_mom),
            )
        stype = 'stage' if stride != 1 else 'normal'
        
        return Bottle2neck(inplanes, planes, stride, downsample, stype=stype)


    def forward(self, x):

        width_output = x.shape[-1] // 8
        height_output = x.shape[-2] // 8
        layers = []

        x = self.conv1(x)

        x = self.layer1(x)
        layers.append(x)

        x = self.layer2(self.relu(x))
        layers.append(x)
  
        x = self.layer3(self.relu(x))
        layers.append(x)
        x_ = self.layer3_(self.relu(layers[1]))

        x = x + self.down3(self.relu(x_))
        x_ = x_ + F.interpolate(
                        self.compression3(self.relu(layers[2])),
                        size=[height_output, width_output],
                        mode='bilinear')
        if self.augment:
            temp = x_

        x = self.layer4(self.relu(x))
        layers.append(x)
        x_ = self.layer4_(self.relu(x_))

        x = x + self.down4(self.relu(x_))
        x_ = x_ + F.interpolate(
                        self.compression4(self.relu(layers[3])),
                        size=[height_output, width_output],
                        mode='bilinear')

        x_ = self.layer5_(self.relu(x_))
        x = self.layer5(self.relu(x))
        
        x = F.interpolate(
                        self.spp(x),
                        size=[height_output, width_output],
                        mode='bilinear')

        x_ = self.final_layer(x + x_)

        if self.augment: 
            x_extra = self.seghead_extra(temp)
            return [x_, x_extra]
        else:
            return x_ 

def DualResNet_imagenet(augument=False):
    model = DualResNet(num_classes=19, planes=64, spp_planes=128, head_planes=128, augment=augument)
    return model

def get_seg_model(augument=False):
    model = DualResNet_imagenet(augument=augument)
    return model

## Preparation steps

In [ ]:
class OhemCrossEntropy(nn.Module):
    def __init__(self, ignore_label=-1, thres=0.7,
                 min_kept=100000, weight=None):
        super(OhemCrossEntropy, self).__init__()
        self.thresh = thres
        self.min_kept = max(1, min_kept)
        self.ignore_label = ignore_label
        self.criterion = nn.CrossEntropyLoss(
            weight=weight,
            ignore_index=ignore_label,
            reduction='none'
        )

    def _ce_forward(self, score, target):
        ph, pw = score.size(2), score.size(3)
        h, w = target.size(1), target.size(2)
        if ph != h or pw != w:
            score = F.interpolate(input=score, size=(
                h, w), mode='bilinear', align_corners=false)

        loss = self.criterion(score, target)

        return loss

    def _ohem_forward(self, score, target, **kwargs):
        ph, pw = score.size(2), score.size(3)
        h, w = target.size(1), target.size(2)
        if ph != h or pw != w:
            score = F.interpolate(input=score, size=(
                h, w), mode='bilinear', align_corners=false)
        pred = F.softmax(score, dim=1)
        pixel_losses = self.criterion(score, target).contiguous().view(-1)
        mask = target.contiguous().view(-1) != self.ignore_label

        tmp_target = target.clone()
        tmp_target[tmp_target == self.ignore_label] = 0
        pred = pred.gather(1, tmp_target.unsqueeze(1))
        pred, ind = pred.contiguous().view(-1,)[mask].contiguous().sort()
        min_value = pred[min(self.min_kept, pred.numel() - 1)]
        threshold = max(min_value, self.thresh)

        pixel_losses = pixel_losses[mask][ind]
        pixel_losses = pixel_losses[pred < threshold]
        return pixel_losses.mean()

    def forward(self, score, target):

        # if config.MODEL.NUM_OUTPUTS == 1:
        #     score = [score]

        weights = [1, 0.4]
        assert len(weights) == len(score)

        functions = [self._ce_forward] * \
            (len(weights) - 1) + [self._ohem_forward]
        return sum([
            w * func(x, target)
            for (w, x, func) in zip(weights, score, functions)
        ])

In [ ]:
class PolynomialLR(_LRScheduler):
    def __init__(self, optimizer, max_iter, decay_iter=1, gamma=0.9, last_epoch=-1):
        self.decay_iter = decay_iter
        self.max_iter = max_iter
        self.gamma = gamma
        super(PolynomialLR, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        factor = (1 - self.last_epoch / float(self.max_iter)) ** self.gamma
        return [base_lr * factor for base_lr in self.base_lrs]

In [ ]:
start_iter = 0
best_iou = -100.0
i = start_iter
flag = True
loss_all = 0
loss_n = 0

In [ ]:
def get_logger(logdir):
    logger = logging.getLogger("DR_test")
    ts = str(datetime.now()).split(".")[0].replace(" ", "_")
    ts = ts.replace(":", "_").replace("-", "_")
    file_path = os.path.join(logdir, "run_{}.log".format(ts))
    hdlr = logging.FileHandler(file_path)
    formatter = logging.Formatter("%(asctime)s %(levelname)s %(message)s")
    hdlr.setFormatter(formatter)
    logger.addHandler(hdlr)
    logger.setLevel(logging.INFO)
    return logger

In [ ]:
!git clone https://github.com/PingoLH/FCHarDNet.git
!cp -r FCHarDNet/ptsemseg ./

Cloning into 'FCHarDNet'...
remote: Enumerating objects: 120, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 120 (delta 0), reused 2 (delta 0), pack-reused 117
Receiving objects: 100% (120/120), 9.09 MiB | 25.58 MiB/s, done.
Resolving deltas: 100% (48/48), done.


In [ ]:
from ptsemseg.loader import get_loader
from ptsemseg.metrics import runningScore, averageMeter
from ptsemseg.augmentations import get_composed_augmentations

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_path = "/content/drive/MyDrive/RnD/runs/DDRNet/HarDBlock_instead_Basic_with_Bottle2neck"
logdir = os.path.join(base_path, str(datetime.fromtimestamp(int(time.time()))))
writer = SummaryWriter(log_dir=logdir)

print("RUNDIR: {}".format(logdir))

logger = get_logger(logdir)
logger.info("Let the games begin")

RUNDIR: /content/drive/MyDrive/RnD/runs/DDRNet/HarDBlock_instead_Basic_with_Bottle2neck/2021-05-25 14:08:31


In [ ]:
# Setup seeds
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)
np.random.seed(1337)
random.seed(1337)

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Setup Augmentations
augmentations = {'hflip': 0.5, 'rscale_crop': [1024, 1024]}
data_aug = get_composed_augmentations(augmentations)

# Setup Dataloader
cfg_data = {
    'dataset': 'cityscapes',
    'train_split': 'train',
    'val_split': 'val',
    'img_rows': 1024,
    'img_cols': 1024,
    'path': '/content/drive/MyDrive/RnD/datasets/',
    'sbd_path': '/content/drive/MyDrive/RnD/datasets/'
}
data_loader = get_loader(cfg_data["dataset"])
data_path = cfg_data["path"]

t_loader = data_loader(
    data_path,
    is_transform=True,
    split=cfg_data["train_split"],
    img_size=(cfg_data["img_rows"], cfg_data["img_cols"]),
    augmentations=data_aug,
)
n_classes = t_loader.n_classes
trainloader = data.DataLoader(
    t_loader,
    batch_size=batch_size,
    num_workers=n_workers,
    shuffle=True,
)

v_loader = data_loader(
    data_path,
    is_transform=True,
    split=cfg_data["val_split"],
    img_size=(1024,2048),
)
valloader = data.DataLoader(
    v_loader, batch_size=batch_size, num_workers=n_workers
)

Found 2975 train images
Found 500 val images


In [ ]:
val_loss_meter = averageMeter()
time_meter = averageMeter()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model_augument = True

In [ ]:
running_metrics_val = runningScore(n_classes)
model = get_seg_model(augument=model_augument).to(device)
total_params = sum(p.numel() for p in model.parameters())
print( 'Parameters:',total_params )
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight)

model = torch.nn.DataParallel(model, device_ids=range(torch.cuda.device_count()))
model.apply(weights_init)
pass

Parameters: 8909486


In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.0005)
lr_scheduler = PolynomialLR(optimizer, max_iter=120000)

In [ ]:
# need model augument
loss_fn = None
if model_augument:
    class_weigth = torch.FloatTensor([  0.8373, 0.918, 0.866, 1.0345, 
                                        1.0166, 0.9969, 0.9754, 1.0489,
                                        0.8786, 1.0023, 0.9539, 0.9843, 
                                        1.1116, 0.9037, 1.0865, 1.0955, 
                                        1.0865, 1.1529, 1.0507]).to(device)
    loss_fn = OhemCrossEntropy(ignore_label=250, thres=0.9, min_kept=131072, weight=class_weigth)
else:
    loss_fn = nn.CrossEntropyLoss(ignore_index=250)

In [ ]:
output_upsample = nn.Upsample((cfg_data["img_rows"], cfg_data["img_cols"]))
output_val_upsample = nn.Upsample((1024, 2048))

In [ ]:
runs = sorted(os.listdir(base_path), reverse=True)
best_path = None
last_path = None
for run in runs:
    tmp_base = os.path.join(base_path, run)
    checkpoint = os.path.join(tmp_base, 'ddrnet_slim_23_cityscapes_checkpoint.pkl')
    best = os.path.join(tmp_base, 'ddrnet_slim_23_cityscapes_best_model.pkl')
    if not last_path and os.path.exists(checkpoint):
        last_path = checkpoint
    if not best_path and os.path.exists(best):
        best_path = best
    if last_path and best_path:
        break

In [ ]:
loaded = torch.load(last_path)
best_iou_arrc = torch.load(best_path)
model_state = loaded.get('model_state')

optimizer_state = loaded.get('optimizer_state')
scheduler_state = loaded.get('lr_scheduler_state')
start_iter = loaded.get('epoch')

best_iou = best_iou_arrc.get('best_iou')
i = start_iter
train_iters = i + 3000 if i + 3000 < max_iter else max_iter
flag = True
loss_all = 0
loss_n = 0

model.load_state_dict(model_state)
optimizer.load_state_dict(optimizer_state)
lr_scheduler.load_state_dict(scheduler_state)

In [ ]:
i, train_iters, best_iou

(101500, 104500, 0.674530616480655)

## Training

In [ ]:
while i <= train_iters and flag:
    for (images, labels, _) in trainloader:
        i += 1
        start_ts = time.time()
        model.train()
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        if model_augument:
            for j in range(len(outputs)):
                outputs[j] = output_upsample(outputs[j])
        else:
            outputs = output_upsample(outputs)
                
        loss = loss_fn(outputs, labels)
        if model_augument:
            loss = loss.mean()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        c_lr = lr_scheduler.get_lr()
 
        time_meter.update(time.time() - start_ts)
        loss_all += loss.item()
        loss_n += 1
        
        if (i + 1) % print_interval == 0:
            fmt_str = "Iter [{:d}/{:d}]  Loss: {:.4f}  Time/Image: {:.4f}    lr={:.6f}"
            print_str = fmt_str.format(
                i + 1,
                train_iters,
                loss_all / loss_n,
                time_meter.avg / batch_size,
                c_lr[0],
            )
            
 
            print(print_str)
            logger.info(print_str)
            writer.add_scalar("loss/train_loss", loss.item(), i + 1)
            time_meter.reset()
 
        if (i + 1) % val_interval == 0 or (i + 1) == train_iters:
            torch.cuda.empty_cache()
            model.eval()
            loss_all = 0
            loss_n = 0
            with torch.no_grad():
                for (images_val, labels_val, _) in valloader:
                    images_val = images_val.to(device)
                    labels_val = labels_val.to(device)
 
                    outputs = model(images_val)
                    if model_augument:
                        for j in range(len(outputs)):
                            outputs[j] = output_val_upsample(outputs[j])
                    else:
                        outputs = output_val_upsample(outputs)
                        # outputs = output_upsample(outputs)
                    val_loss = loss_fn(outputs, labels_val)
                    if model_augument:
                        val_loss = val_loss.mean()
 
                    pred = outputs[0].data.max(1)[1].cpu().numpy()
                    gt = labels_val.data.cpu().numpy()
 
                    running_metrics_val.update(gt, pred)
                    val_loss_meter.update(val_loss.item())
 
            writer.add_scalar("loss/val_loss", val_loss_meter.avg, i + 1)
            logger.info("Iter %d Val Loss: %.4f" % (i + 1, val_loss_meter.avg))
 
            score, class_iou = running_metrics_val.get_scores()
            for k, v in score.items():
                # print(k, v)
                logger.info("{}: {}".format(k, v))
                writer.add_scalar("val_metrics/{}".format(k), v, i + 1)
 
            for k, v in class_iou.items():
                logger.info("{}: {}".format(k, v))
                writer.add_scalar("val_metrics/cls_{}".format(k), v, i + 1)
 
            val_loss_meter.reset()
            running_metrics_val.reset()
            
            state = {
                    "epoch": i + 1,
                    "model_state": model.state_dict(),
                    "optimizer_state": optimizer.state_dict(),
                    "lr_scheduler_state": lr_scheduler.state_dict(),
            }
            save_path = os.path.join(
                writer.file_writer.get_logdir(),
                "{}_{}_checkpoint.pkl".format(model_arch, cfg_data["dataset"]),
            )
            torch.save(state, save_path)
 
            if score["Mean IoU : \t"] >= best_iou:
                best_iou = score["Mean IoU : \t"]
                state = {
                    "epoch": i + 1,
                    "model_state": model.state_dict(),
                    "best_iou": best_iou,
                }
                save_path = os.path.join(
                    writer.file_writer.get_logdir(),
                    "{}_{}_best_model.pkl".format(model_arch, cfg_data["dataset"]),
                )
                torch.save(state, save_path)
            torch.cuda.empty_cache()
 
        if (i + 1) == train_iters:
            flag = False
            break

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


Iter [101510/104500]  Loss: 0.3684  Time/Image: 0.1403    lr=0.001860
Iter [101520/104500]  Loss: 0.3445  Time/Image: 0.1385    lr=0.001860
Iter [101530/104500]  Loss: 0.3423  Time/Image: 0.1373    lr=0.001859
Iter [101540/104500]  Loss: 0.3494  Time/Image: 0.1374    lr=0.001858
Iter [101550/104500]  Loss: 0.3494  Time/Image: 0.1420    lr=0.001857
Iter [101560/104500]  Loss: 0.3500  Time/Image: 0.1378    lr=0.001856
Iter [101570/104500]  Loss: 0.3553  Time/Image: 0.1415    lr=0.001855
Iter [101580/104500]  Loss: 0.3541  Time/Image: 0.1400    lr=0.001854
Iter [101590/104500]  Loss: 0.3553  Time/Image: 0.1410    lr=0.001853
Iter [101600/104500]  Loss: 0.3542  Time/Image: 0.1405    lr=0.001852
Iter [101610/104500]  Loss: 0.3541  Time/Image: 0.1393    lr=0.001851
Iter [101620/104500]  Loss: 0.3582  Time/Image: 0.1411    lr=0.001850
Iter [101630/104500]  Loss: 0.3563  Time/Image: 0.1407    lr=0.001850
Iter [101640/104500]  Loss: 0.3570  Time/Image: 0.1391    lr=0.001849
Iter [101650/104500]

INFO:DR_test:Mean Acc : 	: 0.7767499182079473
INFO:DR_test:FreqW Acc : 	: 0.8936804021198033
INFO:DR_test:Mean IoU : 	: 0.6695965761914783
INFO:DR_test:0: 0.9694074612241906
INFO:DR_test:1: 0.7706282645494541
INFO:DR_test:2: 0.8924764402232047
INFO:DR_test:3: 0.4648900494799621
INFO:DR_test:4: 0.5111853499129001
INFO:DR_test:5: 0.5039168351919051
INFO:DR_test:6: 0.5645487227485759
INFO:DR_test:7: 0.6691871578952446
INFO:DR_test:8: 0.8994873744220818
INFO:DR_test:9: 0.5702188385695229
INFO:DR_test:10: 0.9187457879402068
INFO:DR_test:11: 0.7182319108226106
INFO:DR_test:12: 0.4761553579313314
INFO:DR_test:13: 0.9067999918458521
INFO:DR_test:14: 0.5376100166898361
INFO:DR_test:15: 0.6850845391882346
INFO:DR_test:16: 0.5759082807439221
INFO:DR_test:17: 0.42389238827421893
INFO:DR_test:18: 0.6639601799848324
INFO:DR_test:Iter [102010/104500]  Loss: 0.3389  Time/Image: 0.1551    lr=0.001815


Iter [102010/104500]  Loss: 0.3389  Time/Image: 0.1551    lr=0.001815


INFO:DR_test:Iter [102020/104500]  Loss: 0.3626  Time/Image: 0.1504    lr=0.001814


...

INFO:DR_test:Iter [104490/104500]  Loss: 0.3548  Time/Image: 0.1662    lr=0.001589


Iter [104490/104500]  Loss: 0.3548  Time/Image: 0.1662    lr=0.001589


INFO:DR_test:Iter [104500/104500]  Loss: 0.3541  Time/Image: 0.1694    lr=0.001588


Iter [104500/104500]  Loss: 0.3541  Time/Image: 0.1694    lr=0.001588


INFO:DR_test:Iter 104500 Val Loss: 0.4419
INFO:DR_test:Overall Acc: 	: 0.9404063902140145
INFO:DR_test:Mean Acc : 	: 0.7629994724307663
INFO:DR_test:FreqW Acc : 	: 0.8925960093996492
INFO:DR_test:Mean IoU : 	: 0.6712193270915098
INFO:DR_test:0: 0.9692278552283156
INFO:DR_test:1: 0.7724736929001691
INFO:DR_test:2: 0.8917531275958253
INFO:DR_test:3: 0.46300007902982276
INFO:DR_test:4: 0.42900177267832357
INFO:DR_test:5: 0.4923983793023531
INFO:DR_test:6: 0.5586599118562047
INFO:DR_test:7: 0.6647553757779536
INFO:DR_test:8: 0.8972076671498068
INFO:DR_test:9: 0.5511378011369199
INFO:DR_test:10: 0.9212218751469918
INFO:DR_test:11: 0.7231864335869368
INFO:DR_test:12: 0.4965910774615758
INFO:DR_test:13: 0.909120208542256
INFO:DR_test:14: 0.49624766331072134
INFO:DR_test:15: 0.7063914154039056
INFO:DR_test:16: 0.6462676622001533
INFO:DR_test:17: 0.4919852880630198
INFO:DR_test:18: 0.6725399283674335
